In [2]:
import pandas as pd
import numpy as np
import random
import math

In [46]:
def load_session_file(filePath):
    with open(filePath,"r") as file:
        data = [list(map(int, line.strip().split())) for line in file]
    return data

def init_params():
    
    w_a,w_b,w_c = np.random.dirichlet([1, 1, 1])
    
    p_a = random.uniform(0.2,0.8)
    p_b = random.uniform(0.2,0.8)
    p_c = random.uniform(0.2,0.8)
    
    total = p_a + p_b + p_c
    
    p_a = p_a/total
    p_b = p_b/total
    p_c = p_c/total
    
    return w_a,w_b,w_c,p_a,p_b,p_c

def bernoulli_prob(success,prob_success,n=20):
    
    comb = math.comb(n,success)
    return comb * (prob_success ** success ) * ((1-prob_success) ** (n-success))

def em_min_max(data, max_iter=100, diff=1e-5):
    
    n = len(data)
    flips = 20
    
    w_a,w_b,w_c,p_a,p_b,p_c = init_params()
    
    for curr_iter in range(max_iter):
        
        probs = []
        likelihood = 0
        
        # E-Step
        for session in data:
            
            heads = sum(session)
            prob_a = w_a * bernoulli_prob(heads,p_a)
            prob_b = w_b * bernoulli_prob(heads,p_b)
            prob_c = w_c * bernoulli_prob(heads,p_c)
            
            total_prob = prob_a + prob_b + prob_c 
            
            prob_that_session_was_generated_using_a = prob_a / total_prob
            prob_that_session_was_generated_using_b = prob_b / total_prob
            prob_that_session_was_generated_using_c = prob_c / total_prob
            
            tuple_new = (prob_that_session_was_generated_using_a,prob_that_session_was_generated_using_b,prob_that_session_was_generated_using_c)
            probs.append(tuple_new)
            
            likelihood += math.log(total_prob)
            
        # M-Step Updating Params
        
        w_a = sum(p[0] for p in probs) / n
        w_b = sum(p[1] for p in probs) / n
        w_c = sum(p[2] for p in probs) / n
        
        p_a = sum(r[0] * sum(session) for r, session in zip(probs, data)) / (sum(p[0] for p in probs) * flips)
        p_b = sum(r[1] * sum(session) for r, session in zip(probs, data)) / (sum(p[1] for p in probs) * flips)
        p_c = sum(r[2] * sum(session) for r, session in zip(probs, data)) / (sum(p[2] for p in probs) * flips)

        
        if curr_iter > 0 and abs(likelihood - prev_likelihood) < diff:
            break

        prev_likelihood = likelihood
        
    return w_a,w_b,w_c,p_a,p_b,p_c
    


In [45]:
data = load_session_file('session_data.txt')
w_a,w_b,w_c,p_a,p_b,p_c = em_min_max(data)

print(f"Estimated Mixing Probabilities: π_A={w_a:.3f}, π_B={w_b:.3f}, π_C={w_c:.3f}")
print(f"Estimated Coin Biases: p_A={p_a:.3f}, p_B={p_b:.3f}, p_C={p_c:.3f}")


Estimated Mixing Probabilities: π_A=0.515, π_B=0.307, π_C=0.179
Estimated Coin Biases: p_A=0.610, p_B=0.237, p_C=0.932
